# Surfingcrypto `Portfolio`

The `surfingcrypto` portfolio.

In [161]:
%load_ext autoreload
%autoreload 2
from surfingcrypto import Config
from surfingcrypto.portfolio import Portfolio
from surfingcrypto.portfolio.tracker import Tracker

import pandas as pd
c=Config("/Users/giorgiocaizzi/Documents/GitHub/surfingcrypto/config")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
p=Portfolio("coinbase",configuration=c)

Warning! There are 5 transactionsthat were EXCLUDED in std_df.
Coinbase errors:
TransactionsHistory(Transactions:186 - Processed:186, Unhandled:0 - Errors:9)


In [163]:
p.std_df.head(10)

,type,amount,symbol,native_amount,nat_symbol,total,subtotal,total_fee,spot_price,trade_id
datetime,,,,,,,,,,
2018-10-22T06:29:22Z,buy,0.017237,BTC,100.00,EUR,100.00,97.01,2.99,5627.999719,None
2018-10-22T06:29:22Z,buy,100.000000,EUR,100.00,EUR,NaN,NaN,NaN,1.000000,None
2018-11-09T12:05:12Z,sell,0.017237,BTC,93.27,EUR,93.27,96.26,2.99,5584.488743,None
2018-11-09T12:05:13Z,sell,93.270000,EUR,93.27,EUR,NaN,NaN,NaN,1.000000,None
2018-11-09T12:06:14Z,buy,348.021666,BAT,93.27,EUR,93.27,90.28,2.99,0.259409,None
2018-11-09T12:06:15Z,buy,93.270000,EUR,93.27,EUR,NaN,NaN,NaN,1.000000,None
2019-01-25T14:28:56Z,sell,35.960000,EUR,35.96,EUR,NaN,NaN,NaN,1.000000,None
2019-01-25T14:28:56Z,sell,348.021666,BAT,35.96,EUR,35.96,37.95,1.99,0.109045,None
2021-04-20T18:57:40Z,buy,0.258429,ETH,500.00,EUR,500.00,492.66,7.34,1906.364408,None


In [164]:
p.std_df.total_fee.sum()

234.18000000000026

## Portfolio live value tracker

In [165]:
t=Tracker(p.std_df,stocks_start="1-1-2021",configuration=c)

In [201]:
t.stocks_start

Timestamp('2021-01-01 00:00:00+0000', tz='UTC')

In [194]:
t.portfolio_df.head(20)

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share
0,2021-04-20 18:57:40+00:00,buy,0.258429,ETH,500.00,1906.364408
1,2021-04-22 10:13:53+00:00,buy,0.062128,MKR,250.00,3964.873854
2,2021-04-22 19:20:15+00:00,sell,0.062128,MKR,213.69,3491.496921
3,2021-04-24 08:53:47+00:00,buy,210.343863,ADA,213.69,1.000980
4,2021-04-25 16:15:33+00:00,buy,0.350116,CGLD,1.65,4.712730
5,2021-04-25 16:16:15+00:00,buy,0.350324,CGLD,1.66,4.738470
6,2021-04-25 16:16:40+00:00,buy,0.350324,CGLD,1.66,4.738470
7,2021-04-25 16:20:07+00:00,buy,0.751597,GRT,0.83,1.104315
8,2021-04-25 16:20:47+00:00,buy,0.751597,GRT,0.83,1.104315
9,2021-04-25 16:21:22+00:00,buy,0.751880,GRT,0.83,1.103900


In [168]:
print(len(t.portfolio_df.Symbol.unique()))

27


In [169]:
closedata=t.load_data()
closedata

,Date,Close,symbol
0,2021-01-01 00:00:00+00:00,600.859522,ETH
1,2021-01-02 00:00:00+00:00,638.265609,ETH
2,2021-01-03 00:00:00+00:00,796.329350,ETH
3,2021-01-04 00:00:00+00:00,848.990363,ETH
4,2021-01-05 00:00:00+00:00,894.422659,ETH
...,...,...,...
3037,2022-03-08 00:00:00+00:00,75.478203,SOL
3038,2022-03-09 00:00:00+00:00,79.651404,SOL
3039,2022-03-10 00:00:00+00:00,75.308279,SOL
3040,2022-03-11 00:00:00+00:00,73.982926,SOL


In [170]:
t.error_log

[]

In [171]:
print(len(closedata.symbol.unique()))

27


In [172]:
benchmark=t.set_benchmark("ETH")
benchmark

,Date,Close
0,2021-01-01 00:00:00+00:00,600.859522
1,2021-01-02 00:00:00+00:00,638.265609
2,2021-01-03 00:00:00+00:00,796.329350
3,2021-01-04 00:00:00+00:00,848.990363
4,2021-01-05 00:00:00+00:00,894.422659
...,...,...
431,2022-03-08 00:00:00+00:00,2364.462260
432,2022-03-09 00:00:00+00:00,2465.147372
433,2022-03-10 00:00:00+00:00,2368.094996
434,2022-03-11 00:00:00+00:00,2345.209878


In [173]:
active_portfolio=t.portfolio_start_balance()
active_portfolio

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share
0,2021-04-20 18:57:40+00:00,buy,0.258429,ETH,500.00,1906.364408
1,2021-04-22 10:13:53+00:00,buy,0.062128,MKR,250.00,3964.873854
2,2021-04-22 19:20:15+00:00,sell,0.062128,MKR,213.69,3491.496921
3,2021-04-24 08:53:47+00:00,buy,210.343863,ADA,213.69,1.000980
4,2021-04-25 16:15:33+00:00,buy,0.350116,CGLD,1.65,4.712730
...,...,...,...,...,...,...
163,2022-01-02 23:24:25+00:00,sell,1.026156,SOL,159.04,154.986191
164,2022-01-04 10:33:27+00:00,buy,0.067396,ETH,225.88,3351.534216
165,2022-01-04 10:44:49+00:00,buy,1.511428,SOL,225.73,149.348800
166,2022-01-04 10:44:51+00:00,sell,0.068600,ETH,230.19,3355.559414


In [189]:
active_portfolio.tail(10)

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share
158,2021-12-23 07:27:10+00:00,sell,85.344257,MATIC,199.58,2.338529
159,2021-12-23 07:28:57+00:00,buy,1.001178,AAVE,195.57,195.339958
160,2021-12-23 07:28:58+00:00,sell,85.357006,MATIC,199.82,2.340991
161,2022-01-02 12:28:59+00:00,buy,0.072590,ETH,250.00,3311.907396
162,2022-01-02 12:34:02+00:00,sell,0.071386,ETH,234.98,3291.681842
163,2022-01-02 23:24:25+00:00,sell,1.026156,SOL,159.04,154.986191
164,2022-01-04 10:33:27+00:00,buy,0.067396,ETH,225.88,3351.534216
165,2022-01-04 10:44:49+00:00,buy,1.511428,SOL,225.73,149.348800
166,2022-01-04 10:44:51+00:00,sell,0.068600,ETH,230.19,3355.559414
167,2022-01-04 10:46:31+00:00,sell,1.511428,SOL,226.08,149.580369


In [209]:
print(len(active_portfolio.Symbol.unique()))

27


In [211]:
positions_per_day=t.time_fill(active_portfolio)

In [186]:
pd.set_option('display.max_columns', 500)


In [197]:
positions_per_day[-1]

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share,Date Snapshot


In [156]:
combined_df=t.per_day_portfolio_calcs(
    positions_per_day,
    benchmark,
    closedata,
    t.stocks_start 
)
combined_df

/opt/miniconda3/envs/cryptoenv/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Adj cost,Adj cost per share,Date Snapshot,Open date,Qty,Symbol,Type,Symbol Adj Close,Adj cost daily,Benchmark Close,Benchmark End Date Close,Benchmark Start Date Close,symbol End Date Close,symbol Start Date Close,Equiv Benchmark Shares,Benchmark Start Date Cost,Benchmark Return,symbol Return,symbol Share Value,Benchmark Share Value,Stock Gain / (Loss),Benchmark Gain / (Loss),Abs Value Compare,Abs Value Return,Abs. Return Compare
0,492.660000,1906.364408,2021-04-20 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1936.596366,500.472817,1936.596366,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.223043,0.015858,500.472817,1587.864602,7.812817,1095.204602,7.812817,0.015858,-2.207185
1,492.660000,1906.364408,2021-04-21 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1964.593249,507.708026,1964.593249,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.269638,0.030544,507.708026,1610.819957,15.048026,1118.159957,15.048026,0.030544,-2.239094
2,492.660000,1906.364408,2021-04-22 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,2000.536891,516.996908,2000.536891,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.329459,0.049399,516.996908,1640.291064,24.336908,1147.631064,24.336908,0.049399,-2.280060
3,492.660000,1906.364408,2021-04-23 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1953.619811,504.872170,1953.619811,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.251375,0.024788,504.872170,1601.822557,12.212170,1109.162557,12.212170,0.024788,-2.226587
4,492.660000,1906.364408,2021-04-24 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1828.220570,472.465360,1828.220570,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.042676,-0.040991,472.465360,1499.004530,-20.194640,1006.344530,-20.194640,-0.040991,-2.083667
5,492.660000,1906.364408,2021-04-25 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1915.747276,495.084806,1915.747276,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.188345,0.004922,495.084806,1570.769902,2.424806,1078.109902,2.424806,0.004922,-2.183423
6,492.660000,1906.364408,2021-04-26 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,2098.168223,542.227683,2098.168223,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.491945,0.100612,542.227683,1720.341475,49.567683,1227.681475,49.567683,0.100612,-2.391332
7,492.660000,1906.364408,2021-04-27 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,2202.679525,569.236443,2202.679525,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.665881,0.155435,569.236443,1806.032949,76.576443,1313.372949,76.576443,0.155435,-2.510446
8,492.660000,1906.364408,2021-04-28 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,2263.173707,584.869899,2263.173707,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.766560,0.187167,584.869899,1855.633667,92.209899,1362.973667,92.209899,0.187167,-2.579393
9,492.660000,1906.364408,2021-04-29 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,2273.805980,587.617587,2273.805980,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.784256,0.192745,587.617587,1864.351338,94.957587,1371.691338,94.957587,0.192745,-2.591511


In [157]:
combined_df.columns

Index(['Adj cost', 'Adj cost per share', 'Date Snapshot', 'Open date', 'Qty',
       'Symbol', 'Type', 'Symbol Adj Close', 'Adj cost daily',
       'Benchmark Close', 'Benchmark End Date Close',
       'Benchmark Start Date Close', 'symbol End Date Close',
       'symbol Start Date Close', 'Equiv Benchmark Shares',
       'Benchmark Start Date Cost', 'Benchmark Return', 'symbol Return',
       'symbol Share Value', 'Benchmark Share Value', 'Stock Gain / (Loss)',
       'Benchmark Gain / (Loss)', 'Abs Value Compare', 'Abs Value Return',
       'Abs. Return Compare'],
      dtype='object')

In [158]:
combined_df["Date Snapshot"].unique()

<DatetimeArray>
['2021-04-20 00:00:00+00:00', '2021-04-21 00:00:00+00:00',
 '2021-04-22 00:00:00+00:00', '2021-04-23 00:00:00+00:00',
 '2021-04-24 00:00:00+00:00', '2021-04-25 00:00:00+00:00',
 '2021-04-26 00:00:00+00:00', '2021-04-27 00:00:00+00:00',
 '2021-04-28 00:00:00+00:00', '2021-04-29 00:00:00+00:00',
 '2021-04-30 00:00:00+00:00', '2021-05-01 00:00:00+00:00']
Length: 12, dtype: datetime64[ns, UTC]

In [159]:
combined_df.groupby([combined_df["Date Snapshot"].dt.date]).sum()

,Adj cost,Adj cost per share,Qty,Symbol Adj Close,Adj cost daily,Benchmark Close,Benchmark End Date Close,Benchmark Start Date Close,symbol End Date Close,symbol Start Date Close,Equiv Benchmark Shares,Benchmark Start Date Cost,Benchmark Return,symbol Return,symbol Share Value,Benchmark Share Value,Stock Gain / (Loss),Benchmark Gain / (Loss),Abs Value Compare,Abs Value Return,Abs. Return Compare
Date Snapshot,,,,,,,,,,,,,,,,,,,,,
2021-04-20,492.660000,1906.364408,0.258429,1936.596366,500.472817,1936.596366,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.223043,0.015858,500.472817,1587.864602,7.812817,1095.204602,7.812817,0.015858,-2.207185
2021-04-21,492.660000,1906.364408,0.258429,1964.593249,507.708026,1964.593249,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.269638,0.030544,507.708026,1610.819957,15.048026,1118.159957,15.048026,0.030544,-2.239094
2021-04-22,492.660000,1906.364408,0.258429,2000.536891,516.996908,2000.536891,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.329459,0.049399,516.996908,1640.291064,24.336908,1147.631064,24.336908,0.049399,-2.280060
2021-04-23,492.660000,1906.364408,0.258429,1953.619811,504.872170,1953.619811,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.251375,0.024788,504.872170,1601.822557,12.212170,1109.162557,12.212170,0.024788,-2.226587
2021-04-24,492.660000,1906.364408,0.258429,1828.220570,472.465360,1828.220570,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.042676,-0.040991,472.465360,1499.004530,-20.194640,1006.344530,-20.194640,-0.040991,-2.083667
2021-04-25,492.660000,1906.364408,0.258429,1915.747276,495.084806,1915.747276,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.188345,0.004922,495.084806,1570.769902,2.424806,1078.109902,2.424806,0.004922,-2.183423
2021-04-26,492.660000,1906.364408,0.258429,2098.168223,542.227683,2098.168223,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.491945,0.100612,542.227683,1720.341475,49.567683,1227.681475,49.567683,0.100612,-2.391332
2021-04-27,492.660000,1906.364408,0.258429,2202.679525,569.236443,2202.679525,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.665881,0.155435,569.236443,1806.032949,76.576443,1313.372949,76.576443,0.155435,-2.510446
2021-04-28,492.660000,1906.364408,0.258429,2263.173707,584.869899,2263.173707,2359.128839,600.859522,2359.128839,600.859522,0.819925,492.660000,2.766560,0.187167,584.869899,1855.633667,92.209899,1362.973667,92.209899,0.187167,-2.579393


In [160]:
t.line(combined_df, ["Stock Gain / (Loss)"])
# t.line_facets(combined_df, 'symbol Return', 'Benchmark Return')